In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pymongo
import warnings
import re
warnings.filterwarnings("ignore")
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")
tags_dictionary = client.get(data_as_key)[-1]['tags']
len(tags_dictionary)

6426

In [2]:
bert_data_key = ("ids_to_validate", "data", "bert_tags")
guten_tag_labels = client.get(bert_data_key)[-1]["gutenTag"]    

In [50]:
endDate = datetime(2020, 4, 27)
startDate = datetime.today() 
excess_from_models_counter = 0
excess_from_person_counter = 0
excess_from_person_gutentag_counter = 0
excess_from_person_entity_counter = 0
total_number_of_tags = 0
excess_from_models_gutentag_counter = 0
excess_from_models_entity_counter = 0
validator_remove_collector = []
validator_add_collector = []

validator_remove_BERT_collector = []
validator_remove_ENTITY_collector = []

totalProcessedValidator = db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"}, {"status":"graphicone_feed"},
                                                                          {"status":"validated"}],
                                                                "entity_prepared": {"$exists": True},
                                                                "entity_response_aws_2": {"$exists": True},
                                      'validated_timestamp': {'$gte': endDate, '$lte': startDate}
                                     })

for i in totalProcessedValidator:
    bert_tags = set(i["bert_tags"])
    entity_prepared = set(i["entity_prepared"])
    confirmed_after_validate = set([row.lower() for row in i["confirmed_after_validate"]])
    
    total_number_of_tags+= len(confirmed_after_validate)
    
    validator_remove_BERT_collector += (set(bert_tags) - (set(confirmed_after_validate).intersection(set(guten_tag_labels))))
    validator_remove_ENTITY_collector += (set(entity_prepared) - set(confirmed_after_validate))
#     print(i["entity_response_aws_2"])
    print(i["validator_username"])
    print("bert_tags:",bert_tags)
    print("entity_prepared:",entity_prepared)
    print()
    print("confirmed_after_validate:",confirmed_after_validate)
    print()
    
    excess_from_models = ((bert_tags.union(entity_prepared))- confirmed_after_validate)
    excess_from_models_counter += len(excess_from_models)    
    print("validator_remove",excess_from_models)
    validator_remove_collector += list(excess_from_models)
    
    excess_from_models_gutentag = excess_from_models.intersection(set(guten_tag_labels))
    excess_from_models_gutentag_counter+= len(excess_from_models_gutentag)    
    excess_from_models_entity = excess_from_models - (set(guten_tag_labels))
    excess_from_models_entity_counter+= len(excess_from_models_entity)

    
    
    excess_from_person = (confirmed_after_validate - (bert_tags.union(entity_prepared)))
    excess_from_person_counter += len(excess_from_person)
    
    print("validator_add",excess_from_person)
    validator_add_collector += list(excess_from_person)
    
    excess_from_person_gutentag = excess_from_person.intersection(set(guten_tag_labels))
    excess_from_person_gutentag_counter+= len(excess_from_person_gutentag)
    
    excess_from_person_entity = excess_from_person - (set(guten_tag_labels))
    excess_from_person_entity_counter+= len(excess_from_person_entity)

    
    print()
    print()
    print()
    print()

[{'BeginOffset': 28, 'EndOffset': 34, 'Score': 0.8610521554946899, 'Text': 'google', 'Type': 'G1'}, {'BeginOffset': 37, 'EndOffset': 45, 'Score': 0.9976536631584167, 'Text': 'facebook', 'Type': 'G1'}, {'BeginOffset': 70, 'EndOffset': 78, 'Score': 0.9998393058776855, 'Text': 'facebook', 'Type': 'G1'}, {'BeginOffset': 115, 'EndOffset': 121, 'Score': 0.9938493371009827, 'Text': 'google', 'Type': 'G1'}]
tagger48
bert_tags: set()
entity_prepared: {'google', 'facebook'}

confirmed_after_validate: {'technology', 'government', 'politics', 'google', 'facebook'}

validator_remove set()
validator_add {'government', 'technology', 'politics'}




[{'BeginOffset': 19, 'EndOffset': 24, 'Score': 0.8920394778251648, 'Text': 'brent', 'Type': 'G1'}, {'BeginOffset': 25, 'EndOffset': 28, 'Score': 0.8683324456214905, 'Text': 'oil', 'Type': 'G1'}]
tagger48
bert_tags: {'commodities'}
entity_prepared: {'brent', 'oil', 'commodities'}

confirmed_after_validate: {'brent', 'oil', 'commodities'}

validator_remove s

[{'BeginOffset': 15, 'EndOffset': 20, 'Score': 0.7998028993606567, 'Text': 'trade', 'Type': 'G1'}, {'BeginOffset': 30, 'EndOffset': 36, 'Score': 0.6977614760398865, 'Text': 'dollar', 'Type': 'G1'}, {'BeginOffset': 194, 'EndOffset': 200, 'Score': 0.5072437524795532, 'Text': 'alpine', 'Type': 'G1'}, {'BeginOffset': 201, 'EndOffset': 206, 'Score': 0.5692826509475708, 'Text': 'macro', 'Type': 'G1'}]
tagger48
bert_tags: {'trade'}
entity_prepared: {'currency', 'alpine', 'macro', 'trade', 'usd'}

confirmed_after_validate: {'trade', 'currency', 'usd'}

validator_remove {'macro', 'alpine'}
validator_add set()




[{'BeginOffset': 3, 'EndOffset': 9, 'Score': 0.7993029952049255, 'Text': 'global', 'Type': 'G1'}, {'BeginOffset': 91, 'EndOffset': 97, 'Score': 0.5308195948600769, 'Text': 'abbvie', 'Type': 'G1'}, {'BeginOffset': 141, 'EndOffset': 147, 'Score': 0.7032793760299683, 'Text': 'abbott', 'Type': 'G1'}]
tagger48
bert_tags: {'drugs', 'global', 'pharmaceutical industry'}
entity_prepared: {'abbv



[{'BeginOffset': 5, 'EndOffset': 18, 'Score': 0.816007137298584, 'Text': 'united states', 'Type': 'G1'}, {'BeginOffset': 19, 'EndOffset': 23, 'Score': 0.9758014678955078, 'Text': 'usda', 'Type': 'G1'}, {'BeginOffset': 67, 'EndOffset': 78, 'Score': 0.9457356333732605, 'Text': 'agriculture', 'Type': 'G1'}, {'BeginOffset': 119, 'EndOffset': 123, 'Score': 0.9628468751907349, 'Text': 'pork', 'Type': 'G1'}, {'BeginOffset': 138, 'EndOffset': 145, 'Score': 0.5748399496078491, 'Text': 'chinese', 'Type': 'G1'}, {'BeginOffset': 146, 'EndOffset': 150, 'Score': 0.9475564956665039, 'Text': 'meat', 'Type': 'G1'}, {'BeginOffset': 194, 'EndOffset': 198, 'Score': 0.9625139832496643, 'Text': 'pork', 'Type': 'G1'}, {'BeginOffset': 287, 'EndOffset': 291, 'Score': 0.9328075647354126, 'Text': 'beef', 'Type': 'G1'}, {'BeginOffset': 301, 'EndOffset': 308, 'Score': 0.542701780796051, 'Text': 'chicken', 'Type': 'G1'}, {'BeginOffset': 309, 'EndOffset': 313, 'Score': 0.8786489367485046, 'Text': 'pork', 'Type': '

entity_prepared: {'health', 'art', 'financials', 'warehousing', 'social', 'manufacturing', 'entertainment', 'labor', 'macro', 'retail', 'construction & engineering', 'services', 'leisure', 'trade', 'transportation'}

confirmed_after_validate: {'restaurants', 'entertainment', 'labor', 'services', 'education', 'health', 'financials', 'social', 'manufacturing', 'retail', 'leisure', 'construction & engineering', 'trade', 'travel', 'transportation', 'art', 'information technology', 'employment', 'logistics'}

validator_remove {'macro', 'warehousing'}
validator_add {'restaurants', 'information technology', 'travel', 'logistics'}




[{'BeginOffset': 5, 'EndOffset': 14, 'Score': 0.6958853602409363, 'Text': 'deflation', 'Type': 'G1'}, {'BeginOffset': 42, 'EndOffset': 44, 'Score': 0.5202515721321106, 'Text': 'it', 'Type': 'G1'}, {'BeginOffset': 51, 'EndOffset': 53, 'Score': 0.881137490272522, 'Text': 'us', 'Type': 'G1'}, {'BeginOffset': 54, 'EndOffset': 57, 'Score': 0.5940977334976196, 'Text': 


confirmed_after_validate: {'bankruptcy', 'oneweb', 'softbank', 'wework'}

validator_remove {'wag'}
validator_add {'bankruptcy', 'oneweb'}




[{'BeginOffset': 252, 'EndOffset': 262, 'Score': 0.7516595721244812, 'Text': 'technology', 'Type': 'G1'}]
tagger48
bert_tags: {'labor', 'technology'}
entity_prepared: {'technology'}

confirmed_after_validate: {'technology', 'sustainability', 'data'}

validator_remove {'labor'}
validator_add {'data', 'sustainability'}




[{'BeginOffset': 32, 'EndOffset': 34, 'Score': 0.9349913597106934, 'Text': 'us', 'Type': 'G1'}, {'BeginOffset': 123, 'EndOffset': 129, 'Score': 0.768702507019043, 'Text': 'travel', 'Type': 'G1'}, {'BeginOffset': 137, 'EndOffset': 139, 'Score': 0.9484531283378601, 'Text': 'us', 'Type': 'G1'}]
tagger48
bert_tags: {'travel'}
entity_prepared: {'us', 'travel'}

confirmed_after_validate: {'us', 'travel', 'services'}

validator_remove set()
validator_add {'services'}




[{'BeginOffset': 20, 'EndOffset': 27, 'Score': 0.9112260937690735

In [51]:
total_number_of_tags

6743

In [48]:
print("Total Removed:",excess_from_models_counter/total_number_of_tags)

Total Removed: 0.18448761678777992


In [19]:
print("Total Add:",excess_from_person_counter/total_number_of_tags)

Total Add: 0.254041227940086


In [20]:
1-(excess_from_models_counter/total_number_of_tags)-(excess_from_person_counter/total_number_of_tags)

0.5614711552721341

In [21]:
excess_from_person_counter

1713

In [22]:
# Добавил из гутентага
excess_from_person_gutentag_counter

861

In [23]:
# Добавил из энтити
excess_from_person_entity_counter

852

In [24]:
excess_from_models_counter

1244

In [25]:
excess_from_models_gutentag_counter

476

In [26]:
excess_from_models_entity_counter

768

In [13]:
from collections import Counter

In [73]:
Counter(validator_remove_collector)

Counter({'global': 36,
         'afc': 2,
         'base metals': 1,
         'gc': 1,
         'bayarian': 1,
         'nordic': 1,
         'pharma': 1,
         'mitsubishi': 1,
         'lg': 1,
         'chemical': 2,
         'drugs': 4,
         'vaccines': 1,
         'sina': 2,
         'social science': 1,
         'hackney stockton': 1,
         'trutv': 9,
         'technology': 9,
         'vox media': 2,
         'music': 1,
         'apple': 1,
         'pcs': 1,
         'roku': 1,
         'geopolitics': 1,
         'social': 7,
         'united canada': 1,
         'prime': 10,
         'data': 1,
         'google': 2,
         'exports': 4,
         'real': 1,
         'inbev': 1,
         'all': 1,
         'iwind': 1,
         'batteries': 2,
         'macro': 9,
         'gigi': 2,
         'transportation': 7,
         'oilsands': 2,
         'innovation': 2,
         'rep.': 1,
         'wholesale': 2,
         'construction & engineering': 3,
         'services

In [16]:
len(Counter(validator_remove_collector).most_common())

673

In [52]:
print(len(set(validator_remove_collector)))
len(set(validator_remove_collector) - set(guten_tag_labels))

673


551

In [53]:
set(validator_remove_collector) - set(guten_tag_labels)

{'&p 500',
 'aa',
 'aanlamirhm',
 'abu dhabi investment',
 'ackermann',
 'ad',
 'adam',
 'aemon',
 'afc',
 'agg',
 'ain',
 'airways',
 'aje',
 'alaska',
 'albertsons',
 'albillo',
 'albrecht',
 'alc',
 'all',
 'alloys',
 'almonds',
 'alphonse',
 'alpine',
 'alsaud',
 'altice',
 'alwaleed',
 'americans',
 'americas',
 'amex',
 'amy',
 'ana bot',
 'anna',
 'antibiotics',
 'antivirals',
 'apple',
 'apple inc',
 'apples',
 'arab',
 'asean',
 'asia',
 'astellas',
 'auction',
 'august',
 'authority',
 'avon',
 'backtest',
 'baird piper',
 'baker',
 'bank',
 'bank of china',
 'bank of japan',
 'bankof america',
 'barack obama',
 'barley',
 'barrel',
 'base metals',
 'basrah',
 'batimore',
 'bayarian',
 'bb',
 'bband',
 'bbby',
 'bbgbarc',
 'bea',
 'beats electronics',
 'bebanks',
 'bel',
 'belgio',
 'berkley',
 'bernie',
 'bev',
 'bill',
 'bin talal',
 'bitcoin',
 'bladder',
 'bmi',
 'bonnaf',
 'bos',
 'boston',
 'br',
 'brain',
 'brazilian',
 'breadth',
 'bremen',
 'brian kemp',
 'bristol-my

In [54]:
len(Counter(validator_add_collector).most_common())

655

In [ ]:
#Исключить из предсказаний энтити фиксированные тэги

In [55]:
Counter(validator_remove_ENTITY_collector).most_common()

[('import', 20),
 ('priceline', 19),
 ('global', 18),
 ('usd', 17),
 ('us', 16),
 ('euro', 14),
 ('real estate', 13),
 ('prime', 10),
 ('commodities', 10),
 ('index', 10),
 ('trutv', 9),
 ('retail', 9),
 ('consumer', 8),
 ('currency', 8),
 ('macro', 8),
 ('work', 7),
 ('transportation', 6),
 ('world', 6),
 ('bonds', 6),
 ('virtu', 6),
 ('government bonds', 5),
 ('social', 5),
 ('use', 5),
 ('government', 5),
 ('inflation', 5),
 ('consumer goods', 5),
 ('exports', 4),
 ('financials', 4),
 ('job', 4),
 ('information technology', 4),
 ('s&p', 4),
 ('technology', 3),
 ('services', 3),
 ('smartphone', 3),
 ('internet', 3),
 ('gas', 3),
 ('automotive', 3),
 ('payments', 3),
 ('politics', 3),
 ('shanghai', 3),
 ('trade', 3),
 ('august', 3),
 ('treasury', 3),
 ('finance', 3),
 ('bull', 3),
 ('london', 3),
 ('recession', 3),
 ('bank', 3),
 ('afc', 2),
 ('sina', 2),
 ('vox media', 2),
 ('google', 2),
 ('gigi', 2),
 ('oilsands', 2),
 ('wholesale', 2),
 ('construction & engineering', 2),
 ('tech',

# REMOVED

In [86]:
sum(Counter(validator_remove_collector).values())

1244

# REMOVED BERT FROM ENTITY

In [83]:
removed_entity_bert_tags_counter = {}
removed_entity_all_tags_counter = {}
ddddddddddd = dict(Counter(validator_remove_ENTITY_collector))
for key in ddddddddddd:
    if key in guten_tag_labels:
        removed_entity_bert_tags_counter[key] = ddddddddddd[key]
    else:
        removed_entity_all_tags_counter[key] = ddddddddddd[key]
print(sum(removed_entity_bert_tags_counter.values()))
print(sum(removed_entity_all_tags_counter.values()))

197
768


In [69]:
removed_entity_bert_tags_counter

{'global': 18,
 'chemical': 1,
 'technology': 3,
 'transportation': 6,
 'construction & engineering': 2,
 'services': 3,
 'consumer': 8,
 'debt': 2,
 'credit': 2,
 'mobile': 2,
 'internet': 3,
 'bonds': 6,
 'government bonds': 5,
 'nonfarm payrolls': 1,
 'currency': 8,
 'financials': 4,
 'metals': 2,
 'mining': 1,
 'cities': 1,
 'monetary policy': 1,
 'social': 5,
 'materials': 1,
 'commodities': 10,
 'environment': 1,
 'medicine': 1,
 'real estate': 13,
 'electronics': 1,
 'energy': 1,
 'automotive': 3,
 'delivery': 2,
 'crisis': 2,
 'payments': 3,
 'macro': 8,
 'retail': 9,
 'politics': 3,
 'government': 5,
 'trade': 3,
 'woman': 2,
 'funds': 2,
 'emissions': 1,
 'inflation': 5,
 'information technology': 4,
 'demography': 2,
 'wealth': 1,
 'media': 2,
 'gaming': 1,
 'consumer staples': 1,
 'bear market': 1,
 'nuclear power': 1,
 'mortgage': 1,
 'recession': 3,
 'science': 1,
 'app': 1,
 'population': 1,
 'fuel': 1,
 'wireless': 1,
 'household': 2,
 'taxes': 1,
 'brexit': 1,
 'small 

In [70]:
Counter(validator_remove_BERT_collector).most_common()

[('retail', 25),
 ('global', 23),
 ('diseases', 23),
 ('health', 20),
 ('consumer', 11),
 ('developed markets', 10),
 ('technology', 8),
 ('commodities', 8),
 ('trade', 8),
 ('government', 8),
 ('financials', 8),
 ('consumer discretionary', 7),
 ('consumer products', 6),
 ('energy', 6),
 ('bonds', 6),
 ('interest rates', 5),
 ('medicine', 4),
 ('sentiment', 4),
 ('politics', 4),
 ('leisure', 4),
 ('inflation', 4),
 ('debt', 4),
 ('drugs', 3),
 ('social', 3),
 ('internet', 3),
 ('information technology', 3),
 ('industrials', 3),
 ('government bonds', 3),
 ('funds', 3),
 ('banking', 3),
 ('large caps', 3),
 ('monetary policy', 3),
 ('employment', 3),
 ('batteries', 2),
 ('macro', 2),
 ('innovation', 2),
 ('cloud', 2),
 ('entertainment', 2),
 ('solar energy', 2),
 ('credit', 2),
 ('apparel', 2),
 ('woman', 2),
 ('imports', 2),
 ('transportation', 2),
 ('wind power', 2),
 ('emerging markets', 2),
 ('science', 2),
 ('services', 2),
 ('personal finance', 2),
 ('loan', 2),
 ('crisis', 2),
 ('

In [79]:
removed_bert_bert_tags_counter = {}
ddddddddddd = dict(Counter(validator_remove_BERT_collector))
for key in ddddddddddd:
        removed_bert_bert_tags_counter[key] = ddddddddddd[key]
print(sum(removed_bert_bert_tags_counter.values()))

330


In [87]:
197/1244

0.15836012861736334

In [88]:
330/1244

0.2652733118971061

In [89]:
768/1244

0.617363344051447

In [90]:
197+330+768

1295

In [135]:
endDate = datetime(2020, 4, 27)
startDate = datetime.today() 
excess_from_models_counter = 0
excess_from_person_counter = 0
excess_from_person_gutentag_counter = 0
excess_from_person_entity_counter = 0
total_number_of_tags = 0
excess_from_models_gutentag_counter = 0
excess_from_models_entity_counter = 0
validator_remove_collector = []
validator_add_collector = []

validator_remove_BERT_collector = []
validator_remove_ENTITY_collector = []




EntityBERT_collector_add = []
EntityNoNBERT_collector_add = []
EntityBERT_collector_add = []
BERT_collector_add = []
BothModels_add = []

EntityBERT_collector_remove = []
EntityNoNBERT_collector_remove = []
EntityBERT_collector_remove = []
BERT_collector_remove = []
BothModels_remove = []
Proper_tags_collector = []

totalProcessedValidator = db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"}, {"status":"graphicone_feed"},
                                                                          {"status":"validated"}],
                                                                "entity_prepared": {"$exists": True},
                                                                "entity_response_aws_2": {"$exists": True},
                                      'validated_timestamp': {'$gte': endDate, '$lte': startDate}
                                     })

for i in totalProcessedValidator:
    bert_tags = set(i["bert_tags"])
    entity_prepared = set(i["entity_prepared"])
    confirmed_after_validate = set([row.lower() for row in i["grafeed_confirmed"]])    
    Proper_tags_collector += confirmed_after_validate
    
    EntityBERT_collector_remove_1 = (set(entity_prepared).intersection(set(guten_tag_labels))) - (set(confirmed_after_validate).intersection(set(guten_tag_labels)))        
    
    
    EntityNoNBERT_collector_remove_1 = (set(entity_prepared) - set(guten_tag_labels)) - (set(confirmed_after_validate) - set(guten_tag_labels))
    EntityNoNBERT_collector_add_1 = (set(confirmed_after_validate) - set(guten_tag_labels)) - (set(entity_prepared) - set(guten_tag_labels))
    
    BERT_collector_add_1 = (set(confirmed_after_validate).intersection(set(guten_tag_labels))) - set(bert_tags)
    BERT_collector_remove_1 = set(bert_tags) - (set(confirmed_after_validate).intersection(set(guten_tag_labels)))
    
    EntityBERT_collector_remove += EntityBERT_collector_remove_1
    EntityNoNBERT_collector_add += EntityNoNBERT_collector_add_1
    EntityNoNBERT_collector_remove += EntityNoNBERT_collector_remove_1
    BERT_collector_add += BERT_collector_add_1
    BERT_collector_remove += BERT_collector_remove_1

#     Both_Models_Collector_add = 
    
    
#     print("EntityBERT_collector_add_1:",EntityBERT_collector_add_1)    
    print("EntityBERT_collector_remove_1:", EntityBERT_collector_remove_1)    
    print("EntityNoNBERT_collector_add_1", EntityNoNBERT_collector_add_1)
    print("EntityNoNBERT_collector_remove_1", EntityNoNBERT_collector_remove_1)
    print("BERT_collector_add_1", BERT_collector_add_1)
    print("BERT_collector_remove_1", BERT_collector_remove_1)
#     validator_remove_BERT_collector += (set(bert_tags) - (set(confirmed_after_validate).intersection(set(guten_tag_labels))))
#     validator_remove_ENTITY_collector += (set(entity_prepared) - set(confirmed_after_validate))
#     print(i["entity_response_aws_2"])
#     print(i["validator_username"])
#     print("bert_tags:",bert_tags)
    print()
    print("bert_tags", bert_tags)
    print("entity_prepared:",entity_prepared)
    print("confirmed_after_validate:",confirmed_after_validate)
    print()
    print()
    
#     excess_from_models = ((bert_tags.union(entity_prepared))- confirmed_after_validate)
#     excess_from_models_counter += len(excess_from_models)    
#     print("validator_remove",excess_from_models)
#     validator_remove_collector += list(excess_from_models)
    
#     excess_from_models_gutentag = excess_from_models.intersection(set(guten_tag_labels))
#     excess_from_models_gutentag_counter+= len(excess_from_models_gutentag)    
#     excess_from_models_entity = excess_from_models - (set(guten_tag_labels))
#     excess_from_models_entity_counter+= len(excess_from_models_entity)

    
    
#     excess_from_person = (confirmed_after_validate - (bert_tags.union(entity_prepared)))
#     excess_from_person_counter += len(excess_from_person)
    
#     print("validator_add",excess_from_person)
#     validator_add_collector += list(excess_from_person)
    
#     excess_from_person_gutentag = excess_from_person.intersection(set(guten_tag_labels))
#     excess_from_person_gutentag_counter+= len(excess_from_person_gutentag)
    
#     excess_from_person_entity = excess_from_person - (set(guten_tag_labels))
#     excess_from_person_entity_counter+= len(excess_from_person_entity)

    
#     print()
#     print()
#     print()
#     print()

EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 {'technology', 'politics'}
BERT_collector_remove_1 set()

bert_tags set()
entity_prepared: {'google', 'facebook'}
confirmed_after_validate: {'technology', 'politics', 'google', 'facebook'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 set()
BERT_collector_remove_1 set()

bert_tags {'commodities'}
entity_prepared: {'brent', 'oil', 'commodities'}
confirmed_after_validate: {'brent', 'oil', 'commodities'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 {'afc'}
BERT_collector_add_1 set()
BERT_collector_remove_1 {'global'}

bert_tags {'large caps', 'crisis', 'global'}
entity_prepared: {'s&p', 'us', 'crisis', 'index', 'afc', 'asia'}
confirmed_after_validate: {'s&p', 'us', 'large caps', 'crisis', 'index', 'asia'}


E

EntityBERT_collector_remove_1: {'macro', 'trade'}
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 {'alpine'}
BERT_collector_add_1 {'currency'}
BERT_collector_remove_1 {'trade'}

bert_tags {'trade'}
entity_prepared: {'currency', 'alpine', 'macro', 'trade', 'usd'}
confirmed_after_validate: {'currency', 'usd'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 {'takeda pharmaceutical', 'mylan', 'sun pharmaceutical industries', 'sandoz', 'mayne pharma', 'west ward pharmaceuticals'}
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 set()
BERT_collector_remove_1 set()

bert_tags {'drugs', 'global', 'pharmaceutical industry'}
entity_prepared: {'abbvie', 'global', 'abbott'}
confirmed_after_validate: {'takeda pharmaceutical', 'mylan', 'abbott', 'sun pharmaceutical industries', 'abbvie', 'global', 'drugs', 'sandoz', 'mayne pharma', 'west ward pharmaceuticals', 'pharmaceutical industry'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collecto

bert_tags {'travel'}
entity_prepared: {'macao'}
confirmed_after_validate: {'china', 'travel', 'macao'}


EntityBERT_collector_remove_1: {'inflation'}
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 {'fixed income'}
BERT_collector_remove_1 {'inflation'}

bert_tags {'inflation', 'government bonds', 'bonds'}
entity_prepared: {'inflation', 'government bonds', 'bonds', 'us'}
confirmed_after_validate: {'us', 'government bonds', 'bonds', 'fixed income'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 {'ukraine'}
EntityNoNBERT_collector_remove_1 {'barley', 'ukrain'}
BERT_collector_add_1 set()
BERT_collector_remove_1 {'retail'}

bert_tags {'export', 'agriculture', 'commodities', 'retail', 'trade'}
entity_prepared: {'barley', 'middle east', 'corn', 'export', 'ukrain', 'commodities', 'wheat', 'trade'}
confirmed_after_validate: {'middle east', 'corn', 'export', 'agriculture', 'commodities', 'ukraine', 'wheat', 'trade'}


EntityBE

bert_tags {'banking'}
entity_prepared: {'deutsche bank', 'banking'}
confirmed_after_validate: {'deutsche bank', 'banking'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 set()
BERT_collector_remove_1 set()

bert_tags {'loan', 'financials'}
entity_prepared: {'us', 'loan'}
confirmed_after_validate: {'us', 'loan', 'financials'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 {'default'}
BERT_collector_remove_1 {'debt', 'credit', 'financials'}

bert_tags {'debt', 'loan', 'credit', 'financials'}
entity_prepared: {'us', 'loan'}
confirmed_after_validate: {'us', 'loan', 'default'}


EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 {'financials'}
BERT_collector_remove_1 set()

bert_tags {'loan', 'bonds', 'fixed income', 'high yield'}
entity

KeyError: 'grafeed_confirmed'

In [136]:
total_number_of_tags = sum(dict(Counter(Proper_tags_collector)).values())
print("Proper_tags_collector", total_number_of_tags)
print("-------------------------")
print("EntityBERT_collector_remove",sum(dict(Counter(EntityBERT_collector_remove)).values()), round(sum(dict(Counter(EntityBERT_collector_remove)).values())/total_number_of_tags,3)) 
print("EntityNoNBERT_collector_add", sum(dict(Counter(EntityNoNBERT_collector_add)).values()), round(sum(dict(Counter(EntityNoNBERT_collector_add)).values())/total_number_of_tags,3)) 
print("EntityNoNBERT_collector_remove", sum(dict(Counter(EntityNoNBERT_collector_remove)).values()),round(sum(dict(Counter(EntityNoNBERT_collector_remove)).values())/total_number_of_tags,3)) 
print("BERT_collector_add", sum(dict(Counter(BERT_collector_add)).values()), round(sum(dict(Counter(BERT_collector_add)).values())/total_number_of_tags,3))
print("BERT_collector_remove", sum(dict(Counter(BERT_collector_remove)).values()),round(sum(dict(Counter(BERT_collector_remove)).values())/total_number_of_tags,3))
print("-------------------------")

Proper_tags_collector 4685
-------------------------
EntityBERT_collector_remove 152 0.032
EntityNoNBERT_collector_add 580 0.124
EntityNoNBERT_collector_remove 577 0.123
BERT_collector_add 746 0.159
BERT_collector_remove 369 0.079
-------------------------


In [ ]:
добавить meat 'lamb', 'beef', 'gas', 'fish', 'commodities', 'nuts', 'chicken', 'cheese', 'bebanks', 'retail', 'pork', 'tofu'

In [129]:
0.029+0.126+0.114+(0.175+0.049)

0.493

In [137]:
(0.159 + 0.079)

0.238

In [138]:
0.124+0.123

0.247

In [ ]:
# алгоритмы похожести прочекать

In [139]:
pd.DataFrame.from_dict(Counter(EntityNoNBERT_collector_add).most_common()).to_excel("ahaha2.xlsx")

In [ ]:
EntityBERT_collector_remove 152 0.032
EntityNoNBERT_collector_add 580 0.124
EntityNoNBERT_collector_remove 577 0.123
BERT_collector_add 746 0.159
BERT_collector_remove 369 0.079

#23 процента ошибок берта
#24 процента ошибок энтити